In [4]:
!pip install --upgrade --quiet google-genai

In [5]:
!pip install gTTS --quiet

In [6]:
!pip install playsound --quiet

  Preparing metadata (setup.py) ... done


In [7]:
from google.colab import userdata
GEMINI_API_KEY: str = userdata.get('GOOGLE_API_KEY_1')
if(GEMINI_API_KEY):
  print("API Key found")
else:
  print("API Key not found")

API Key found


In [8]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str = "gemini-2.0-flash-exp"

In [9]:
video_file = "/content/2fa5eddc7dda45ca830680ab825d0a50.mp4"

In [19]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

my_video = upload_video(video_file)

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/c23wki35ipbu


In [20]:
from google.genai.types import Content, Part
prompt = """For each scene in this video,
            generate captions that describe the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = my_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)

print(response.text)

```json
[
    {
        "timecode": "0:00",
        "caption": "Two individuals are standing close to each other, and text overlayed on the video says, \"CAUSE YOU ARE\"."
    },
    {
        "timecode": "0:01",
         "caption": "Two individuals stand beneath several hanging light fixtures; text overlayed on the video says, \"THAT SOMEONE\"."
    },
     {
        "timecode": "0:02",
        "caption": "One individual has their arm around the shoulders of another individual, and text overlayed on the video says, \"THAT GETS ME\"."
    },
 {
        "timecode":"0:04",
        "caption":"Two individuals are holding hands and moving; text overlayed on the video says, \"LIKE NO ONE ELSE RIGHT\"."
    },
 {
       "timecode":"0:06",
       "caption":"Two individuals are standing facing the same direction holding hands; text overlayed on the video says, \"WHEN I NEED IT\"."
    },
  {
        "timecode":"0:08",
        "caption":"Three individuals are laughing and holding hands, and text

In [26]:
 tts = gTTS(text=response.text, lang='en')

with open('test.mp3', 'wb') as f:
  for chunk in tts.stream():
    f.write(chunk)

display(Audio("test.mp3", autoplay=True)) # Corrected filename here